# 2 - BBS with ECDSA Holder Binding

While BBS allows to present parts of the Verifiable Credential in an anonymous
and unlinkable manner, there are limits.
The [RFC](https://datatracker.ietf.org/doc/draft-irtf-cfrg-bbs-signatures/) only
supports selective disclosure.
This works for fields which have a big anonymity set, like the NPA, or the canton of
residence.
However, for more linkable fields, like the birthdate, full name, the anonymity set
gets very small and can drop to 1.
An anonymity set of 1 means that there is only one VC with that field, so it can
be linked between usages.

To achieve holder binding, it is necessary to sign each presentation of a VC
with the private key of the device.
For the verification part, the verifier usually needs the public key.
However, this public key is unique.
Even worse, the holder binding needs to be done for every presentation of the VC,
so all presentations can be linked!

One way to solve this is to create many VCs for a user, each VC with a corresponding
public key created by the Secure Element of the wallet, e.g., the phone.
This is not an ideal solution for the following reasons: 

- it increases load on the server who has to issue VCs on a regular basis
- to increase privacy, these must be renewed, even if the VCs are not used
- some secure elements have a limit of how many private keys they can store

Several other solutions have been proposed so far:

- BBS# from Orange - which is missing a scientific paper
[Presentation](https://csrc.nist.gov/csrc/media/presentations/2024/wpec2024-3b3/images-media/wpec2024-3b3-slides-antoine-jacques--BBS-sharp-eIDAS2.pdf)
- Google ZKP, presented at Funke 24 - but in June '25 there is still no code available
- Ubique proposed a solution based on [ZKAttest](https://eprint.iacr.org/2021/1183) - with code available, and based on a peer-reviewed paper from Cloudflare

## ZKAttest from Cloudflare

The ZKAttest paper from Cloudflare presents an interesting solution to create ZKPs
on operations for the P-256 curve.
In short, they use a new elliptic curve with properties allowing it to create short
proofs on operations on the P-256 curve.
This new curve is introduced in the ZKAttest paper and called `T-256`, or `Tom-256`.
While the solution from Google is very generic and works even with an example
using mDoc, the `T-256` curve is only useful if the VC is based on BBS(+).

As with SD-JWT, the holder gets their VC from the issuer:

1. The holder proves their identity, creates a keypair with their Secure Element, and sends the public key to the issuer
2. The issuer creates a VC with all value of the fields hashed, including the public key from the holder
3. The issuer signs the VC using the its secret key, and sends it to the holder
4. The holder stores this VC in their device

Unlike the SD-JWT, the BBS VC can be 'blinded', but still be verified using the public
key from the issuer.

## 2.1 - Setting up the systems

In a first step, you need to create the three main actors of this exercise:

- `Issuer` represents the servers run by the Swiss government, and which can issue credentials representing a person.
For this exercise we don't consider other issuers, like ticket issuers, diploma issuers, or others.
The issuer has a certificate, its public key, which can be used by the verifier to check if the holder presents a
valid proof.
- `Holder` is the user who wants to participate in the E-ID system. Technically, the holder is represented by a mobile
phone, which has the `secure_element()` and the `swiyu()` application. The holder only has access to the public key
of the `secure_element()`, which makes all of this work necessary to prove they have a valid credential.
- `Verifier` requests a proof from the holder that they have been verified by the issuer. In this exercise we show
how this proof cannot been linked between two presentations, as all its fields are different.

A fourth element is already initialized: the `Setup` represents publicly known cryptographic data which must be shared
between the issuer, the holder, and the verifier. It needs to be passed to each of the three main actors, so that
they have the same setup.

!! Rust being rust, we need to mark the variables as `mut`, so they can change their values later in the exercises:

```rust
let mut issuer = ...
```

In [ ]:
:dep ./ecdsa_proof

use ecdsa_proof::*;

println!("Setting up systems");
let setup = PublicSetup::new();
let mut issuer = Issuer::new(setup.clone());
let mut holder = MobilePhone::new(setup.clone());
let mut verifier = Verifier::new(setup.clone(), issuer.get_certificate());

### Discussions

- What is in the `PublicSetup`, and which parts are created by whom?
- What can go wrong with this setup, and how is the system protected?
- 

## 2.2 - Installing the Swiyu app and adding a credential

Now we use the `holder` variable, which represents the mobile phone, and 'install' the Swiyu app.
The installation is done in a much more manual way than a standard user will be doing.
The goal here is to show the necessary steps and the data exchange between the different actors.

1. Install the Swiyu-app
2. Create a keypair from the `secure_element()`
3. Fetch a new credential from the `issuer`, passing it the public key
4. Adding the credential to the `swiyu()` app

In [ ]:
// Install the Swiyu app and add a credential
println!("Creating a credential");
holder.install_swiyu();
let se_kp = holder.secure_element().create_kp();
let credential = issuer.new_credential(se_kp.key_pub);
holder.swiyu().add_vc(se_kp.id, credential);

## 2.3 - Requesting a proof from the holder

Now that the holder has their credential, they are ready to use it.
For our very simple example, we suppose that a verifier wants to request information from a holder.
The verifier starts with a random message, to make sure that the proof cannot be used in another
context.
Here are the different steps necessary for the proof generation by the holder:

1. The `verifier` creates a new, random, message and sends it to the holder
2. The `holder` uses their `swiyu()` to create a `blinded_presentation` of their credential
3. Then the `holder` creates a signature on the `message` sent by the `verifier`
4. Finally the `holder` creates a proof that:
  - they know a public key `p` which can be used to verify the signature
  - this public key `p` is represented in the `blinded_presentation` of the credential

In [ ]:
// Verifier requests a presentation from the holder
println!("Requesting a presentation");
let message = verifier.create_message();

// Holder creates a blinded presentation, an ECDSA signature, and a proof.
// This is of course all done in the Swiyu app, but here we do it step-by-step.
println!("Creating a presentation and an ECDSA proof");
let presentation = holder.swiyu().blinded_presentation(&message, &[]);
let signature: ecdsa::Signature = holder.secure_element().sign(se_kp.id, &message);
let proof = ECDSAProof::new(setup, se_kp.key_pub, presentation.clone(), signature, &message);
let presentation_closed = presentation.close();

## 2.4 - Verifying the proof

Now the `verifier` gets the proof, and can verify that:
- it is a valid signature by the `issuer`
- it has been signed by the public key hidden in the presentation

In [ ]:
// Holder sends the proof to the verifier, which checks it.
println!("Verify the ECDSA proof");
verifier.check_proof(&message, presentation_closed.clone(), proof.clone());

println!("All good!");

## 2.5 - More exercises

1) Verifying the signatures
  - Check that multiple presentations give unlinkable representations
  - Create another `verifier.create_message` and make sure that the verification fails
  - Create another `holder` and make sure that the different signature fails
2) Diving into the cryptography
  - The `BBSPresentation` structure has two fields for the commitments - what is the `Open` and `Close` state?
  - Look at the `ECDSAProof` structure to see which other commitment and which proofs are used
  - What are the verifications done in the `verifier`?
  - Which verification is still missing?
3) Revealing messages in the BBS Protocol
  - The `VerifiedCredential` has two fields which represent a `String`:
    - Change the second parameter to `blinded_presentation` by passing it one or both of
      - `VerifiedCredential::FIELD_FIRSTNAME`
      - `VerifiedCredential::FIELD_LASTNAME`
    - Then, after the `verifier.check_proof`, print the corresponding field using
      `presentation_closed.message_string`
  - Where is the verification made that these `String`s match?
  - What does it change with respect to the unlinkability to reveal data from the BBS Proof?

## 2.6 - Links

- [BBS IRTF Draft](https://datatracker.ietf.org/doc/draft-irtf-cfrg-bbs-signatures/)
- 